In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import math

In [5]:
df = pd.read_csv('Data laboratorio 1/students.csv', sep=';')
df.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In [6]:
df['Target'].unique()

array(['Dropout', 'Graduate', 'Enrolled'], dtype=object)

In [7]:
#Unificando Graduate + Enrolled
df['Target']=df['Target'].replace('Graduate','Enrolled')
df['Target'].unique()

array(['Dropout', 'Enrolled'], dtype=object)

In [8]:
#Separando en train test



In [9]:
#Tipos de datos, buscando los datos continuos
pd.set_option('display.max_rows', None)
df.dtypes

Marital status                                      int64
Application mode                                    int64
Application order                                   int64
Course                                              int64
Daytime/evening attendance\t                        int64
Previous qualification                              int64
Previous qualification (grade)                    float64
Nacionality                                         int64
Mother's qualification                              int64
Father's qualification                              int64
Mother's occupation                                 int64
Father's occupation                                 int64
Admission grade                                   float64
Displaced                                           int64
Educational special needs                           int64
Debtor                                              int64
Tuition fees up to date                             int64
Gender        

In [10]:
print(df.select_dtypes(include=['float']).columns)

Index(['Previous qualification (grade)', 'Admission grade',
       'Curricular units 1st sem (grade)', 'Curricular units 2nd sem (grade)',
       'Unemployment rate', 'Inflation rate', 'GDP'],
      dtype='object')


#### Preprocesing
* Unificar los valores graduate enrolled de Target
* Pasar los datos continuos a low mid y high (según aplique)
    * Previous Qualification
    * Admission Grade
    * Curricular units 1st sem (grade) - (no dice continuous en la pagina pero float)
    * Curricular units 2nd sem (grade) -    "
    * Unemployment rate
    * Inflation rate
    * GDP


In [11]:
#Pasando a categorías los calificaciones previas
df['Previous qualification (grade)'] = pd.cut(df['Previous qualification (grade)'], bins=3, labels=["low",'mid','high'], right=False)


In [12]:
df['Previous qualification (grade)'].unique()

['low', 'high', 'mid']
Categories (3, object): ['low' < 'mid' < 'high']

In [13]:
columns = df.select_dtypes(include=['float']).columns
for column in columns:
    df[column] = pd.cut(df[column], bins=3, labels=["low",'mid','high'], right=False)


In [19]:
df[df["Target"]=="Enrolled"].shape[0]

3003

#### ID3

In [36]:
#Calculo de la entropía total
def entropia(data, etiqueta, valores):
    entropia = [0] * len(valores)
    for i, val in enumerate(valores):
        ci = data[data[etiqueta]==val].shape[0]
        tot = data.shape[0]
        entropia[i]= -ci/tot * math.log(ci/tot)
    return sum(entropia)




In [37]:
#Entropía total
entropia(df, "Target", df["Target"].unique())

0.6277725714164744

In [43]:
#Entropía del subconjunto
entropia(df[df["GDP"]=="high"],"Target", df["Target"].unique() )


0.6021107924130149